# Sequential Model
A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-07-16 16:09:12.441931: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 16:09:12.658542: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 16:09:12.660094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 16:09:13.932454: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
model = keras.Sequential([
    layers.Dense(4, activation="relu", name="layer1"),
    layers.Dense(2, activation="softmax", name="layer2")
])

In [13]:
x = tf.ones((3, 4))

When you instantiate a Sequential model without an **input shape**, it isn't "built": it has no weights.

In [15]:
try:
    model.summary()
except Exception as e:
    print(e)

This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.


In [16]:
model(x)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.4773456 , 0.52265435],
       [0.4773456 , 0.52265435],
       [0.4773456 , 0.52265435]], dtype=float32)>

In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (3, 4)                    20        
                                                                 
 layer2 (Dense)              (3, 2)                    10        
                                                                 
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


This is equivalent to **y = layers.Dense_2(layers.Dense_4(x))**

A Sequential model is not appropriate when:
   - Your model has multiple inputs or multiple outputs
   - Any of your layers has multiple inputs or multiple outputs
   - You need to do layer sharing
   - You want non-linear topology (e.g. a residual connection, a multi-branch model)


In [9]:
model.layers

### Pass Input Layer

- ***It can be very useful when building a Sequential model incrementally to be able to display the summary of the model so far, including the current output shape. In this case, you should start your model by passing an Input object to your model, so that it knows its input shape from the start.***

In [24]:
model = model = keras.Sequential([
    layers.Input((5)),
    layers.Dense(4, activation="relu", name="layer1"),
    layers.Dense(2, activation="softmax", name="layer2")
])

In [25]:
model.summary() #not this won't throw any error

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 4)                 24        
                                                                 
 layer2 (Dense)              (None, 2)                 10        
                                                                 
Total params: 34
Trainable params: 34
Non-trainable params: 0
_________________________________________________________________


In [27]:
x = tf.ones((3, 5))
model(x)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.6965787 , 0.30342135],
       [0.6965787 , 0.30342135],
       [0.6965787 , 0.30342135]], dtype=float32)>

###  Tips: Common debugging workflow add() + summary()
- When building a new Sequential architecture, it's useful to incrementally stack layers with add() and frequently print model summaries. For instance, this enables you to monitor how a stack of Conv2D and MaxPooling2D layers is downsampling image feature maps

### What to do once you have a model
- Once your model architecture is ready, you will want to:
- Train your model, evaluate it, and run inference. See our guide to training & evaluation with the built-in loops
- Save your model to disk and restore it. See our guide to serialization & saving.
- Speed up model training by leveraging multiple GPUs. See our guide to multi-GPU and distributed training.


# Feature extraction with a Sequential model
- **Once a Sequential model has been built, it behaves like a Functional API model. This means that every layer has an input and output attribute.** 
- These attributes can be used to do neat things, like quickly creating a model that extracts the outputs of all intermediate layers in a Sequential model

In [114]:
model = keras.Sequential([
    layers.Input((32, 32, 3)),
    layers.Conv2D(filters=32, kernel_size= 5, activation="relu", name="layer1"),
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", name="intermediate"),
    layers.Flatten(name="flatten"),
    layers.Dense(2, activation="softmax", name="layer2")
])

In [115]:
x = tf.ones((1, 32, 32, 3))
model(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.44129378, 0.5587063 ]], dtype=float32)>

In [102]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Conv2D)             (1, 28, 28, 32)           2432      
                                                                 
 intermediate (Conv2D)       (1, 24, 24, 32)           25632     
                                                                 
 flatten (Flatten)           (1, 18432)                0         
                                                                 
 layer2 (Dense)              (1, 2)                    36866     
                                                                 
Total params: 64,930
Trainable params: 64,930
Non-trainable params: 0
_________________________________________________________________


In [103]:
# return all the layers of the model
model.layers

In [104]:
# return the input tensor
model.inputs

[<KerasTensor: shape=(1, 32, 32, 3) dtype=float32 (created by layer 'layer1_input')>]

- **KerasTensor**: KerasTensors are used within the Keras API to define the inputs, outputs, and intermediate layers of a neural network model.
- while a regular TensorFlow tensor is a general-purpose tensor used in TensorFlow computations, a KerasTensor is a specialized tensor object used within the Keras API, providing additional capabilities such as automatic shape inference and automatic differentiation.

In [105]:
model.outputs

[<KerasTensor: shape=(1, 2) dtype=float32 (created by layer 'layer2')>]

We can also check the input and output tensor of any layers

In [106]:
model.get_layer(name="intermediate").input

<KerasTensor: shape=(1, 28, 28, 32) dtype=float32 (created by layer 'layer1')>

In [107]:
model.get_layer(name="intermediate").output

<KerasTensor: shape=(1, 24, 24, 32) dtype=float32 (created by layer 'intermediate')>

In [108]:
# here you can see, the output of one layer is the input of the layer just below of that layer
model.get_layer(name="layer2").input,  model.get_layer(name="flatten").output

(<KerasTensor: shape=(1, 18432) dtype=float32 (created by layer 'flatten')>,
 <KerasTensor: shape=(1, 18432) dtype=float32 (created by layer 'flatten')>)

let's create a feature_extractor model that includes the layers upto intermediate layer of our main model

In [109]:
feature_extractor = keras.Model(
    inputs=model.inputs,
    outputs=model.get_layer(name="intermediate").output
)

In [110]:
feature_extractor.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1_input (InputLayer)   [(1, 32, 32, 3)]          0         
                                                                 
 layer1 (Conv2D)             (1, 28, 28, 32)           2432      
                                                                 
 intermediate (Conv2D)       (1, 24, 24, 32)           25632     
                                                                 
Total params: 28,064
Trainable params: 28,064
Non-trainable params: 0
_________________________________________________________________


In [111]:
x = tf.ones((1, 32, 32, 3))
out = feature_extractor(x)

In [112]:
out.shape

TensorShape([1, 24, 24, 32])

In [113]:
out[0][0][0][:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.        , 0.        , 0.3494678 , 0.22146514, 0.05251887,
       0.        , 0.        , 0.32228705, 0.        , 0.        ],
      dtype=float32)>